# Ataque de adversario

En este cuaderno, voy a intentar realizar un ataque de adversario sobre una red neuronal construida utilizando Keras/Tensorwlow.

Un ataque de adversario consiste en conseguir una imagen que, sin parecer manipulada, una red neuronal la clasifica erróneamente. Con esto, se podría llegar a provocar que, por ejemplo, un coche autónomo al ver una señal de STOP la identifique como una señal de aumento de velocidad, provocando una situación peligrosa.

## Evaluación de la imagen

En primer lugar, voy a realizar una evaluación normal de una imagen de un coche. Para ello, primero importamos las librerías que necesitamos.

In [1]:
import tensorflow as tf # Importamos la librería de Tensorflow
import keras # Importamos Keras que trabaja sobre Tensor facilitándonos el trabajo
import matplotlib.pyplot as plt # Importamos Matplotlib para poder mostrar gráficas
import numpy as np # Importamos la librería Numpy para trabajar con matrices

Using TensorFlow backend.


Ahora, vamos a cargar nuestro modelo precargado utilizando Keras.

In [2]:
# Importamos los modelos entrenados de Keras, en especial la InceptionV3, de Google y para imágenes, así como, 
# decode_predictions que nos servirá luego para saber que respuesta nos ha dado la red neuronal
from keras.applications.inception_v3 import InceptionV3, decode_predictions
from keras import backend as K
from keras.preprocessing import image # Para trabajar con procesado de imágenes

A continuación, vamos a cargar el modelo de la librería.

In [ ]:
iv3 = InceptionV3()

Finalmente, vamos a utilizar la imagen para probar nuestro modelo, en este caso, será la imagen de un coche.

In [ ]:
# Cargamos la imagen en una variable como array con un tamaño redimensionado a 299x299
X = image.img_to_array(image.load_img("./coche.png", target_size=(299,299))) 

# InceptionV3 utiliza un formato diferente, el rango de intensidades va de -1 a 1. Vamos a reescalar este rango.
# Pasamos de [0,256) -> [-1,1]

X /= 255 
X -= 0.5 
X *= 2 

La red neuronal nos pide que pasemos un formato especial, un tensor, que tiene unas dimensiones parecidas
a las que tenemos ahora, pero necesitamos una dimensión más:

In [ ]:
X = X.reshape([1, X.shape[0], X.shape[1], X.shape[2]])

Ahora, vamos a predecir qué tenemos en la imagen:

In [ ]:
y = iv3.predict(X)

Pero, vamos a ver qué ha predicho:

In [ ]:
decode_predictions(y)

La imagen que le hemos pasado es la siguiente, que nos ha dicho que es un coche deportivo:

In [ ]:
# Vamos a reescalar primero el rango de intensidades a como estaba originalmente
X1 = np.copy(X)

X1 /= 2
X1 += 0.5
X1 *= 255

In [ ]:
plt.imshow(X1[0].astype(np.uint8))
plt.show()

## Generando la imagen para realizar el ataque de adversario

Ahora vamos a generar la nueva imagen, la que provocará que la red neuronal se equivoque. Esto es, ir provocando que se maximice el error:

In [ ]:
input_layer = iv3.layers[0].input
output_layer = iv3.layers[-1].output

# Vamos a hacer creer a la red que queremos que lo clasifique como un limón (número 951)
target_class = 951

# Creamos una función de coste para que la última capa el resultado que muestre sea 'target_class'
loss = output_layer[0, target_class]

Ahora vamos a realizar el gradiente, para encontrar el mínimo, es decir, el objetivo:

In [ ]:
# Nos hace el gráfo entre el valor de entrada y el cálculo del gradiente
grad = K.gradients(loss, input_layer)[0]

# Función que nos hace el cálculo del gradiente
optimize_gradient = K.function([input_layer, K.learning_phase()], [grad, loss])

Una vez tenemos esto, hacemos el bucle donde ejecutar este código:

In [ ]:
adv = np.copy(X)

cost = 0.0

while cost < 0.95:
    gr, cost = optimize_gradient([adv, 0]) # Utilizamos la copia que realizamos anteriormente
    adv += gr # Manipulamos los píxeles

Finalmente, vamos a devolver al rango de intesidades la nueva imagen y a mostrarla.

In [ ]:
adv /= 2
adv += 0.5
adv *= 255

In [ ]:
plt.imshow(adv[0].astype(np.uint8))
plt.show()

Ahora vemos la imagen hackeada, y se ven ciertos pixeles que nos pueden hacer sospechar. Además, vamos a guardar dicha imagen para que se persistan.

In [ ]:
# Guardamos la imagen
from PIL import Image
im = Image.fromarray(adv[0].astype(np.uint8))
im.save("./coche_hacked.png")

Vamos a cargar de nuevo la imagen y a realizar la predicción (limón):

In [ ]:
# Cargamos la imagen en una variable como array con un tamaño redimensionado a 299x299
img_hacked = image.img_to_array(image.load_img("./coche_hacked.png")) 

# InceptionV3 utiliza un formato diferente, el rango de intensidades va de -1 a 1. Vamos a reescalar este rango.
# Pasamos de [0,256) -> [-1,1]

img_hacked /= 255 
img_hacked -= 0.5 
img_hacked *= 2 

img_hacked = img_hacked.reshape([1, img_hacked.shape[0], img_hacked.shape[1], img_hacked.shape[2]])
y = iv3.predict(img_hacked)

decode_predictions(y)

### Minimizar la diferencia entre imágenes

Ahora vamos a decirle a la red neuronal que no se extralimite haciendo la manipulación, así la imagen pasará por una imagen normal.

In [ ]:
adv = np.copy(X)

# Límite de la perturbación
pert = 0.01

max_pert = X + pert
min_pert = X - pert

cost = 0.0

while cost < 0.95:
    gr, cost = optimize_gradient([adv, 0]) # Utilizamos la copia que realizamos anteriormente
    adv += gr # Manipulamos los píxeles
    adv = np.clip(adv, min_pert, max_pert) # Ponemos el límite a la perturbación
    adv = np.clip(adv, -1, 1) # Ponemos el límite a la intensidad

Vamos a mostrar el resultado, para ver que la imagen parece que no ha sido alterada:

In [ ]:
adv /= 2
adv += 0.5
adv *= 255

In [ ]:
plt.imshow(adv[0].astype(np.uint8))
plt.show()

Si realizamos la predicción de nuevo, veremos como nos dice que lo que hay en la imagen es un limón:

In [ ]:
im = Image.fromarray(adv[0].astype(np.uint8))
im.save("./coche_hacked_good.png")

# Cargamos la imagen en una variable como array con un tamaño redimensionado a 299x299
img_hacked = image.img_to_array(image.load_img("./coche_hacked_good.png")) 

# InceptionV3 utiliza un formato diferente, el rango de intensidades va de -1 a 1. Vamos a reescalar este rango.
# Pasamos de [0,256) -> [-1,1]

img_hacked /= 255 
img_hacked -= 0.5 
img_hacked *= 2 

img_hacked = img_hacked.reshape([1, img_hacked.shape[0], img_hacked.shape[1], img_hacked.shape[2]])
y = iv3.predict(img_hacked)

decode_predictions(y)